In [1]:
import tqdm
import json
import requests
import os
from dotenv import load_dotenv 

load_dotenv()

BASE_URL = "https://api.the-odds-api.com/v4"
API_KEY = os.environ.get("ODDS_API_KEY")

REGIONS = ["eu"] # Betting regions
MARKETS = ["h2h"] # Betting markets to choose from 
SPORT = "basketball_nba_preseason" # Sport to interact with. Choose in the print sports cell.

THRESHOLD = 0.05 # # Accounts for percentage losses from vigorish

/Users/nathanzhao/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def get_sports(key) -> list:
    url = f"{BASE_URL}/sports/"

    response = requests.get(url, params= {
        "apiKey" : key
    })

    return list({sport["key"] for sport in response.json()})

def get_data(key, sport, regions, markets) -> list:

    url = f"{BASE_URL}/sports/{sport}/odds/"

    response = requests.get(url, params={
        "apiKey" : key,
        "regions" : ",".join(regions),
        "markets" : ",".join(markets),
        "oddsFormat" : "decimal" # American = +/-100 * (fraction odds of winning or losing, whichever >= 1). Sign based on win/loss

        # finding select bookmakers with frequent mispricings would improve arbitrage finding rates
        # "bookmakers" : TBA
    })

    print('Remaining requests', response.headers['x-requests-remaining'])
    print('Used requests', response.headers['x-requests-used'])
    
    return response.json()

In [9]:
def filter_arbitrages(data, markets, threshold):
    MAX_VAL = 10**9

    arbitrages = {}
    odds_counts = {}

    if "h2h" in markets:
        odds_counts["h2h"] = (MAX_VAL) * 3 # WIN, LOSE, DRAW --- some sports have draws others don't?
    
    for match in data:
        for bookmaker in match["bookmakers"]:
            for market in bookmaker["markets"]:
                market_name = market["key"]
                if market_name == "h2h":
                    print(market)
                    implied_odds = sum(1/outcome["price"] for outcome in market["outcomes"])
                
                if implied_odds < 1 - threshold:

                    match_name = f'{match["home_team"]} vs. {match["away_team"]}'


                    if implied_odds > arbitrages.get(match_name, (-1, -1, -1))[-1]:
                        arbitrages[match_name] = {"bookmaker" : bookmaker["key"], "market" : market["key"], "odds" : implied_odds}

    return arbitrages

In [4]:
sports = get_sports(API_KEY)

print(sports) # See possible sports to choose from

['soccer_uefa_europa_league', 'icehockey_nhl_championship_winner', 'soccer_netherlands_eredivisie', 'soccer_brazil_campeonato', 'soccer_poland_ekstraklasa', 'soccer_norway_eliteserien', 'soccer_usa_mls', 'soccer_mexico_ligamx', 'soccer_italy_serie_b', 'soccer_finland_veikkausliiga', 'soccer_austria_bundesliga', 'soccer_germany_bundesliga', 'basketball_euroleague', 'rugbyunion_world_cup', 'soccer_uefa_euro_qualification', 'soccer_spain_la_liga', 'soccer_epl', 'soccer_belgium_first_div', 'soccer_france_ligue_two', 'basketball_nba_championship_winner', 'soccer_australia_aleague', 'soccer_england_league1', 'basketball_wnba', 'soccer_sweden_superettan', 'soccer_england_league2', 'basketball_ncaab', 'soccer_japan_j_league', 'soccer_spain_segunda_division', 'soccer_uefa_europa_conference_league', 'cricket_icc_world_cup', 'icehockey_sweden_hockey_league', 'soccer_england_efl_cup', 'soccer_germany_bundesliga2', 'soccer_denmark_superliga', 'golf_masters_tournament_winner', 'soccer_uefa_champs_le

In [5]:
pricings_data = get_data(API_KEY, SPORT if SPORT else sports[0], REGIONS, MARKETS)

Remaining requests 473
Used requests 27


In [10]:
arbitrages = filter_arbitrages(pricings_data, MARKETS, THRESHOLD) 
print(arbitrages)

{'key': 'h2h', 'last_update': '2023-10-16T05:36:42Z', 'outcomes': [{'name': 'Atlanta Hawks', 'price': 2.08}, {'name': 'Indiana Pacers', 'price': 1.76}]}
{'key': 'h2h', 'last_update': '2023-10-16T05:36:53Z', 'outcomes': [{'name': 'Atlanta Hawks', 'price': 2.03}, {'name': 'Indiana Pacers', 'price': 1.8}]}
{'key': 'h2h', 'last_update': '2023-10-16T05:36:42Z', 'outcomes': [{'name': 'Brooklyn Nets', 'price': 2.1}, {'name': 'Philadelphia 76ers', 'price': 1.72}]}
{'key': 'h2h', 'last_update': '2023-10-16T05:36:53Z', 'outcomes': [{'name': 'Brooklyn Nets', 'price': 2.09}, {'name': 'Philadelphia 76ers', 'price': 1.75}]}
{'key': 'h2h', 'last_update': '2023-10-16T05:36:42Z', 'outcomes': [{'name': 'Houston Rockets', 'price': 1.53}, {'name': 'San Antonio Spurs', 'price': 2.5}]}
{'key': 'h2h', 'last_update': '2023-10-16T05:36:53Z', 'outcomes': [{'name': 'Houston Rockets', 'price': 1.57}, {'name': 'San Antonio Spurs', 'price': 2.43}]}
{'key': 'h2h', 'last_update': '2023-10-16T05:36:53Z', 'outcomes': [

In [19]:
if len(arbitrages) == 0:
    print(f"No arbitrages found for threshold {THRESHOLD} in sport {SPORT if SPORT else sports[0]}.")

for match_name in arbitrages:
    details = arbitrages[match_name]
    print(f'Match: {match_name}, Bookmaker: {details["bookmaker"]}, \
        Market: {details["market"]}, Total Implied Odds: {details["odds"]}')

Match: Philadelphia 76ers vs. Boston Celtics, Bookmaker: betfair_ex_eu,         Market: h2h_lay, Total Implied Odds: 0.9117693900302596
Match: Los Angeles Lakers vs. Sacramento Kings, Bookmaker: betfair_ex_eu,         Market: h2h_lay, Total Implied Odds: 0.9202195018995357
